In [7]:
import sqlite3
import csv
import pandas as pd

In [50]:
conn = sqlite3.connect('database_click_rate.db') # Create Database
cursor = conn.cursor()

In [51]:
# Create Campaigns table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Campaigns (
        campaign_id INTEGER PRIMARY KEY,
        campaign_name TEXT UNIQUE,
        drug_name TEXT,
        medic_group TEXT,
        adv_format TEXT
    )
''')

# Create Metrics table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Metrics (
        metric_id INTEGER PRIMARY KEY AUTOINCREMENT,
        campaign_id INTEGER,
        date TEXT,
        impressions INTEGER,
        clicks INTEGER,
        click_rate REAL,
        campaign_number INTEGER,
        trend TEXT,
        growth REAL,
        FOREIGN KEY (campaign_id) REFERENCES Campaigns(campaign_id)
    )
''')



In [52]:
# Read CSV file
with open('click_stats.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)  # Skip header
    campaign_name_mapping = {}  # To map campaign_name to campaign_id

    for row in csv_reader:
        campaign_name = row[0]
        drug_name = row[5]
        medic_group = row[6]
        adv_format = row[7]
        impressions = (row[2])
        clicks = (row[3])
        click_rate = (row[4])
        campaign_number = (row[8])
        trend = row[9]
        growth = (row[10])

        # Check if campaign_name is already mapped to a campaign_id
        if campaign_name not in campaign_name_mapping:
            cursor.execute('''
                INSERT INTO Campaigns (campaign_name, drug_name, medic_group, adv_format)
                VALUES (?, ?, ?, ?)
            ''', (campaign_name, drug_name, medic_group, adv_format))
            campaign_id = cursor.lastrowid
            campaign_name_mapping[campaign_name] = campaign_id
        else:
            campaign_id = campaign_name_mapping[campaign_name]

        cursor.execute('''
            INSERT INTO Metrics (campaign_id, date, impressions, clicks, click_rate, campaign_number, trend, growth)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', (campaign_id, row[1], impressions, clicks, click_rate, campaign_number, trend, growth))

In [55]:
cursor.execute("SELECT * FROM Metrics")
metrics_data = cursor.fetchall()
for row in metrics_data:
    print(row)

In [57]:
# Commit changes and close the connection
conn.commit()
conn.close()